In [30]:
# import packages

import pandas as pd
import numpy as np
from docplex.mp.model import Model

objective: 8.500
status: OPTIMAL_SOLUTION(2)
  "x name"=1
  "y name"=0.500
  "z name"=2


In [31]:
sm = Model(name="Sudoku Model")

# Constants
TOTAL_ROWS = 9
TOTAL_COLUMNS = 9
TOTAL_VALUES = 9
SUM_OF_NUMBERS = 45
centre_coordinates = []

# appends the list with the coordinates of the centres of the 9 different 3 x 3 grids 
for a in range(1, 9, 3):
    for b in range(1, 9, 3):
        centre_coordinates.append((a, b))

# Variables
x = {(i, j): sm.integer_var(name = "Row %d, Column %d" % (i + 1, j + 1)) for i in range(TOTAL_ROWS) for j in range(TOTAL_COLUMNS)}
p = {(i, j, k): sm.binary_var(name = "Value %d present in Row %d, Column % d" % (k, i + 1, j + 1)) for i in range(TOTAL_ROWS) for j in range(TOTAL_COLUMNS) for k in range(1, TOTAL_VALUES + 1)}

# In any row, there are no repeats in numbers
for i in range(TOTAL_ROWS):
    for k in range(1, TOTAL_VALUES + 1):
        sm.add_constraint(sm.sum(p[i, j, k] for j in range(TOTAL_COLUMNS)) == 1)

# In any column, there are no repeats in numbers
for j in range(TOTAL_COLUMNS):
    for k in range(1, TOTAL_VALUES + 1):
        sm.add_constraint(sm.sum(p[i, j, k] for i in range(TOTAL_ROWS)) == 1)

# In any 3 x 3 grid, there are no repeats in numbers
for t in centre_coordinates:
    row_index = t[0]
    column_index = t[1]
    for k in range(1, TOTAL_VALUES + 1):
        sm.add_constraint(sm.sum(p[i, j, k] for i in range(row_index - 1, row_index + 2) for j in range(column_index - 1, column_index + 2)) == 1)

for i in range(TOTAL_ROWS):
    for j in range(TOTAL_COLUMNS):
        # binds the two variables together and ensures 1 <= x <= 9
        sm.add_constraint(x[i, j] == sm.sum(k * p[i, j, k] for k in range(1, TOTAL_VALUES + 1)))

        # ensures each cell has exactly one integer value
        sm.add_constraint(sm.sum(p[i, j, k] for k in range(1, TOTAL_VALUES + 1)) == 1)



In [32]:
def populate_grid(sm, ls_cells):
    total_val = 0

    for item in ls_cells:
        row_index = item[0]
        column_index = item[1]
        val = item[2]
        total_val += val

        sm.add_constraint(x[row_index, column_index] == val)
        for k in range(1, 10):
            if k == val:
                sm.add_constraint(p[row_index, column_index, k] == 1)
            else:
                sm.add_constraint(p[row_index, column_index, k] == 0)
        
    return sm

# Hard Level, NY Times, 11 Oct 2019
ls_hard = [(0, 2, 9), (0, 3, 7), (0, 8, 3), (1, 3, 9), (1, 6, 1), (2, 3, 3), (2, 5, 6), (2, 8, 8), (3, 0, 9), (3, 2, 6), (3, 5, 4), (4, 0, 2), (4, 2, 3), (4, 5, 5), (4, 8, 6), (5, 7, 5), (5, 8, 7), (6, 1, 3), (6, 5, 2), (6, 7, 8), (6, 8, 5), (7, 0, 8), (8, 0, 1)]

# Extreme Level
ls_extreme = [(0, 2, 8), (0, 3, 9), (0, 4, 4), (1, 0, 9), (1, 2, 2), (1, 5, 8), (1, 6, 5), (2, 1, 7), (2, 8, 6), (3, 1, 6), (3, 2, 5), (3, 5, 2), (3, 7, 3), (4, 3, 6), (4, 5, 7), (5, 1, 9), (5, 3, 3), (5, 6, 6), (5, 7, 4), (6, 0, 8), (6, 7, 5), (7, 2, 1), (7, 3, 5), (7, 6, 4), (7, 8, 8), (8, 4, 2), (8, 5, 4), (8, 6, 1)]

sm = populate_grid(sm, ls_extreme)

        

In [33]:
# # Clues (Easy Level)
# total_val = 0
# cell = [0, 3]
# val = 8
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [0, 4]
# val = 2
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [0, 6]
# val = 7
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [1, 4]
# val = 6
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [1, 7]
# val = 9
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [1, 8]
# val = 5
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [2, 0]
# val = 7
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [2, 5]
# val = 5
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [3, 2]
# val = 8
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [3, 3]
# val = 5
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [3, 5]
# val = 1
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [3, 8]
# val = 9
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [4, 0]
# val = 2
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [4, 2]
# val = 5
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [4, 6]
# val = 4
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [4, 8]
# val = 1
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [5, 0]
# val = 1
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [5, 3]
# val = 9
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [5, 5]
# val = 2
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [5, 6]
# val = 8
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [6, 3]
# val = 6
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [6, 8]
# val = 3
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [7, 0]
# val = 4
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [7, 1]
# val = 2
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [7, 4]
# val = 5
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [7, 2]
# val = 1
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [7, 4]
# val = 9
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

# cell = [7, 5]
# val = 8
# total_val += val
# sm.add_constraint(x[cell[0], cell[1]] == val)
# for k in range(1, 10):
#     if k == val:
#         sm.add_constraint(p[cell[0], cell[1], k] == 1)
#     else:
#         sm.add_constraint(p[cell[0], cell[1], k] == 0)

In [36]:
sm.solve()
# sm.print_solution()

for i in range(TOTAL_ROWS):
    line_output = ""
    for j in range(TOTAL_COLUMNS):
        line_output += str(int(x[i, j].solution_value))
    print(line_output)

sm._has_solution


618945723
932768514
574213986
165492837
483657291
297381645
849176352
721539468
356824179


<bound method Model._has_solution of docplex.mp.Model['Sudoku Model']>